<a href="https://colab.research.google.com/github/ITU-Business-Analytics-Team/Business_Analytics_for_Professionals/blob/main/Part%20I%20%3A%20Methods%20%26%20Technologies%20for%20Business%20Analytics/Chapter%205%3A%20Neural%20Networks%20and%20Deep%20Learning/5_2_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural Networks and Deep Learning**
## Deep Learning

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

## Problem Definition
Garment employees productivty dataset is used for the following deep learning examples. In the dataset attributes given in the below table is used to predict the productivity of the employees measured between 0-1$^1$. 

$^1$ Imran, Abdullah Al, Md Shamsur Rahim, and Tanvir Ahmed. "Mining the productivity data of the garment industry." International Journal of Business Intelligence and Data Mining 19.3 (2021): 319-342.

| Column Name | Description|
|-------------|------------|
|date |Date in MM-DD-YYYY|
|day  |Day of the Week|
|quarter   |A portion of the month. A month was divided into four quarters|
|department   |Associated department with the instance|
| team  |Associated team number with the instance |
| no_of_workers  |Number of workers in each team |
|  no_of_style_change |Number of changes in the style of a particular product |
| targeted_productivity  |Targeted productivity set by the Authority for each team for each day. |
| smv  |Standard Minute Value, it is the allocated time for a task |
| wip  | Work in progress. Includes the number of unfinished items for products|
| over_time  |Represents the amount of overtime by each team in minutes |
| incentive  |Represents the amount of financial incentive (in BDT) that enables or motivates a particular course of action. |
|idle_time   |The amount of time when the production was interrupted due to several reasons |
| idle_men  | The number of workers who were idle due to production interruption|
|actual_productivity |The actual % of productivity that was delivered by the workers. It ranges from 0-1.|


In [ ]:
df = pd.read_csv("garments_worker_productivity.csv")
df.head()

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382


First lets preprocess the data by converting `team` variable into categorical data, dropping the `date` column, removing the rows with NAs and converting categorical variables into numeric ones.

In [ ]:
df["team"] = df["team"].astype("str")
df.drop("date", axis = 1, inplace=True)
df.dropna(inplace=True)
df = pd.get_dummies(df, drop_first = True)

Next specify the inputs and outputs, split the data into training and testing sets and normalize the data

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop("actual_productivity", axis = 1)
y = df["actual_productivity"]

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)

mu = X_train.mean(axis = 0)
sigma = X_train.std(axis = 0)


X_train = (X_train - mu) / sigma
X_test = (X_test - mu) / sigma

print(X_train.shape)

(483, 29)


 # Deep learning model for regression
Prepare the deep learning model for training, since we are doing regression model we have used one neuron in the output layer with linear activation function defined as:
$$f(x) = x$$

In [ ]:
mdl = tf.keras.Sequential()

mdl.add(tf.keras.layers.Dense(64, activation="relu", input_shape = (X_train.shape[1],)))
mdl.add(tf.keras.layers.Dense(32, activation="relu"))
mdl.add(tf.keras.layers.Dense(16, activation="relu"))
mdl.add(tf.keras.layers.Dense(1,activation="linear"))
mdl.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1920      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 4,545
Trainable params: 4,545
Non-trainable params: 0
_________________________________________________________________


For regression models we genereally use `mean squared error(mse)` metric as loss function.
$$mse = \frac{1}{n}\sum_{i=1}^n(y-\hat{y})^2$$

Also we are going to monitor `mean absolute percenate error(mape)` value for every iteration

$$mape = \frac{1}{n}\sum_{i=1}^n\frac{|y-\hat{y}|}{|y|}*100$$




In [ ]:
mdl.compile(optimizer="adam", loss = "mse", metrics=["mape"])
mdl.fit(X_train,y_train, epochs=10)

Epoch 1/10
16/16 [==============================] - 1s 2ms/step - loss: 0.3915 - mape: 74.5864
Epoch 2/10
16/16 [==============================] - 0s 2ms/step - loss: 0.0777 - mape: 35.8410
Epoch 3/10
16/16 [==============================] - 0s 2ms/step - loss: 0.0436 - mape: 26.1042
Epoch 4/10
16/16 [==============================] - 0s 2ms/step - loss: 0.0307 - mape: 21.5333
Epoch 5/10
16/16 [==============================] - 0s 3ms/step - loss: 0.0245 - mape: 19.3565
Epoch 6/10
16/16 [==============================] - 0s 4ms/step - loss: 0.0207 - mape: 17.5937
Epoch 7/10
16/16 [==============================] - 0s 3ms/step - loss: 0.0176 - mape: 16.3759
Epoch 8/10
16/16 [==============================] - 0s 4ms/step - loss: 0.0155 - mape: 15.3296
Epoch 9/10
16/16 [==============================] - 0s 3ms/step - loss: 0.0138 - mape: 14.3861
Epoch 10/10
16/16 [==============================] - 0s 3ms/step - loss: 0.0122 - mape: 13.6424


## Deep Learning for Binomial Classification
Now we would like to setup a deep learning model for classification. For that purpose we are going to label employees that are over 70% productivity as 1 and others as 0.

In [ ]:
X = df.drop("actual_productivity", axis = 1)
y = (df["actual_productivity"] > 0.7).astype("int")

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)

mu = X_train.mean(axis = 0)
sigma = X_train.std(axis = 0)


X_train = (X_train - mu) / sigma
X_test = (X_test - mu) / sigma


For binary classification we need to set the activation function in the last layer as `sigmoid` which is defined as

$$f(x) = \frac{1}{1 + e^{-x}}$$

In [ ]:
mdl = tf.keras.Sequential()

mdl.add(tf.keras.layers.Dense(64, activation="relu", input_shape = (X_train.shape[1],)))
mdl.add(tf.keras.layers.Dense(32, activation="relu"))
mdl.add(tf.keras.layers.Dense(16, activation="relu"))
mdl.add(tf.keras.layers.Dense(1,activation="sigmoid"))
mdl.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                1920      
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 16)                528       
                                                                 
 dense_15 (Dense)            (None, 1)                 17        
                                                                 
Total params: 4,545
Trainable params: 4,545
Non-trainable params: 0
_________________________________________________________________


Loss function should be binary crossentorpy
$$L = \frac{1}{n}\sum_{i=1}^ny_ilog(\hat{p}_i)+(1-y_i)log(1-\hat{p}_i)$$
 and we can monitor the metric accuracy
$$accuracy = \frac{number\;of\; correct\; labeled}{number\; observed}$$

In [ ]:
mdl.compile(optimizer="adam", loss = "binary_crossentropy", metrics=["accuracy"])
mdl.fit(X_train,y_train, epochs=10)

Epoch 1/10
16/16 [==============================] - 1s 2ms/step - loss: 0.6058 - accuracy: 0.7288
Epoch 2/10
16/16 [==============================] - 0s 2ms/step - loss: 0.5103 - accuracy: 0.7433
Epoch 3/10
16/16 [==============================] - 0s 2ms/step - loss: 0.4412 - accuracy: 0.8012
Epoch 4/10
16/16 [==============================] - 0s 2ms/step - loss: 0.3727 - accuracy: 0.8551
Epoch 5/10
16/16 [==============================] - 0s 2ms/step - loss: 0.3111 - accuracy: 0.8882
Epoch 6/10
16/16 [==============================] - 0s 3ms/step - loss: 0.2613 - accuracy: 0.9110
Epoch 7/10
16/16 [==============================] - 0s 3ms/step - loss: 0.2218 - accuracy: 0.9337
Epoch 8/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1958 - accuracy: 0.9337
Epoch 9/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1753 - accuracy: 0.9358
Epoch 10/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1559 - accuracy: 0.9400


# Deep Learning for Multinomial Classification

Using `k means clustering` function in `sklearn` library we are going to divide the output values into three categories.

In [ ]:
from sklearn.cluster import KMeans
cluster = KMeans(3).fit(df["actual_productivity"].values.reshape(-1,1))


X = df.drop("actual_productivity", axis = 1)
y = cluster.predict(df["actual_productivity"].values.reshape(-1,1))

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)

mu = X_train.mean(axis = 0)
sigma = X_train.std(axis = 0)


X_train = (X_train - mu) / sigma
X_test = (X_test - mu) / sigma

For multinomial classification; you need same number of neurons in the output layer as the number of categories in your target variable. Activation function should be selected as `softmax`.

$$softmax(x) = \frac{e^{-x_i}}{\sum e^{-x_i}}$$

In [ ]:
mdl = tf.keras.Sequential()

mdl.add(tf.keras.layers.Dense(64, activation="relu", input_shape = (X_train.shape[1],)))
mdl.add(tf.keras.layers.Dense(32, activation="relu"))
mdl.add(tf.keras.layers.Dense(16, activation="relu"))
mdl.add(tf.keras.layers.Dense(3,activation="softmax"))
mdl.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 64)                1920      
                                                                 
 dense_17 (Dense)            (None, 32)                2080      
                                                                 
 dense_18 (Dense)            (None, 16)                528       
                                                                 
 dense_19 (Dense)            (None, 3)                 51        
                                                                 
Total params: 4,579
Trainable params: 4,579
Non-trainable params: 0
_________________________________________________________________


If your target variables is one hot encoded you can use `categorical_crossentropy`, if it is label encoded you can use `sparse_categorical_crossentropy` as your loss function

In [ ]:
mdl.compile(optimizer="adam", loss = "sparse_categorical_crossentropy", metrics=["accuracy"])
mdl.fit(X_train,y_train, epochs=10)

Epoch 1/10
16/16 [==============================] - 0s 2ms/step - loss: 1.1434 - accuracy: 0.3644
Epoch 2/10
16/16 [==============================] - 0s 2ms/step - loss: 0.9998 - accuracy: 0.4762
Epoch 3/10
16/16 [==============================] - 0s 2ms/step - loss: 0.9197 - accuracy: 0.6108
Epoch 4/10
16/16 [==============================] - 0s 2ms/step - loss: 0.8366 - accuracy: 0.6563
Epoch 5/10
16/16 [==============================] - 0s 2ms/step - loss: 0.7436 - accuracy: 0.7184
Epoch 6/10
16/16 [==============================] - 0s 2ms/step - loss: 0.6549 - accuracy: 0.7702
Epoch 7/10
16/16 [==============================] - 0s 2ms/step - loss: 0.5746 - accuracy: 0.7867
Epoch 8/10
16/16 [==============================] - 0s 2ms/step - loss: 0.5091 - accuracy: 0.8137
Epoch 9/10
16/16 [==============================] - 0s 2ms/step - loss: 0.4544 - accuracy: 0.8323
Epoch 10/10
16/16 [==============================] - 0s 2ms/step - loss: 0.4193 - accuracy: 0.8468
